In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

In [ ]:
fig = plt.figure()
plt.plot(np.sin(np.linspace(0, 20, 100)));

In [ ]:
# Always hide the toolbar
# fig.canvas.toolbar_visible = False
# Hide the Figure name at the top of the figure
fig.canvas.header_visible = False
# Disable the resizing feature
fig.canvas.resizable = False
# If true then scrolling while the mouse is over the canvas will not move the entire notebook
fig.canvas.capture_scroll = True

In [ ]:
display(fig.canvas)

---
---
# Layouting

In [ ]:
from mpl_toolkits.mplot3d import axes3d
X, Y, Z = axes3d.get_test_data(0.05)

In [ ]:
import ipywidgets as widgets

with plt.ioff():
    fig = plt.figure()

ax = fig.gca()
ax.imshow(Z)

widgets.AppLayout(
    center=fig.canvas,
    footer=widgets.Button(icon='check'),
    pane_heights=[0, 6, 1]
)

---
---
# Performance

In [ ]:
# precomputing all images
x = np.linspace(0,np.pi,200)
y = np.linspace(0,10,200)
X,Y = np.meshgrid(x,y)
parameter = np.linspace(-5,5)
example_image_stack = np.sin(X)[None,:,:]+np.exp(np.cos(Y[None,:,:]*parameter[:,None,None]))

In [ ]:
with plt.ioff():
    fig = plt.figure()
im = plt.imshow(example_image_stack[0])

In [ ]:
def update(change):
    im.set_data(example_image_stack[change['new']])
    fig.canvas.draw_idle()

In [ ]:
slider = widgets.IntSlider(value=0, min=0, max=len(parameter)-1)
slider.observe(update, names='value')
widgets.VBox([slider, fig.canvas])

---
---
# Drawing multiple lines at once

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import AppLayout

In [ ]:
C = 2
x = np.linspace(0, 20, 500)
data = np.stack((np.sin(x), np.cos(x)))

In [ ]:
fig, ax = plt.subplots()
ax.set_ylim([-1,1])
fig.canvas.header_visible = False
fig.canvas.resizable = False

In [ ]:
lines = None
i = 5
if lines is None:
    lines = ax.plot(x[:i], data[:,:i].T)
    fig.canvas.draw()

In [ ]:
for i in range(5, len(x), 5):
    for line, d in zip(lines, data):
        line.set_data(x[:i], d[:i])
    ax.autoscale_view(False, True)
    fig.canvas.draw()
    plt.pause(1e-5)
        # fig.canvas.flush_events()
# else:
#     lines = None

In [ ]:
for i in range(5, len(x), 5):
    ax.clear()
    ax.plot(data[:i])
    fig.canvas.draw()

---
---
# OOP

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import AppLayout

In [ ]:
class Displayer(AppLayout):
    def __init__(self, C: int = 1) -> None:
        with plt.ioff():
            self.fig = plt.figure()
        self.fig.canvas.resizable = False
        self.fig.canvas.header_visible = False
        self.ax = self.fig.gca()
        self.ax.set_ylim([-1,1])
        self.ax.set_xlim([0, 300])
        self.ax.autoscale(False, 'y')
        self.ax.set_title('Probabilities Over Time')
        self.lines = self.ax.plot(np.arange(0,2), np.ones((C, 2)))
        super().__init__(center=self.fig.canvas)
        
    def plot(self, x) -> None:
        x_ax = np.arange(x.shape[0])
        if hasattr(self, "lines"):
            for line, y in zip(self.lines, x.T):
                line.set_data(x_ax, y)
        else:
            self.lines = self.ax.plot(x)
        
        update_axes = len(x_ax) > self.ax.get_xlim()[1]
        if update_axes:
            self.ax.set_xlim([0, int(1.5 * self.ax.get_xlim()[1] )])
            self.fig.canvas.draw()
        # self.fig.canvas.flush_events()
    
    def clear(self) -> None:
        for line in self.lines:
            self.ax.lines.remove(line)
    
    def __del__(self):
        plt.close(self.fig)

In [ ]:
disp = Displayer(C=2)
disp

In [ ]:
x = np.zeros((2,500))
x[0] = np.sin(np.linspace(0, 20, 500))
x[1] = np.cos(np.linspace(0, 20, 500))

In [ ]:
for i in range(5, x.shape[1], 10):
    # disp.plot(x[:][:i])
    disp.plot(x[:,:i].T)

---

In [ ]:
# When using the `widget` backend from ipympl,
# fig.canvas is a proper Jupyter interactive widget, which can be embedded in
# an ipywidgets layout. See https://ipywidgets.readthedocs.io/en/stable/examples/Layout%20Templates.html

# One can bound figure attributes to other widget values.
from ipywidgets import AppLayout, FloatSlider

plt.ioff()

slider = FloatSlider(
    orientation='horizontal',
    description='Factor:',
    value=1.0,
    min=0.02,
    max=2.0
)

slider.layout.margin = '0px 30% 0px 30%'
slider.layout.width = '40%'

fig = plt.figure()
fig.canvas.header_visible = False
fig.canvas.layout.min_height = '400px'
plt.title('Plotting: y=sin({} * x)'.format(slider.value))

x = np.linspace(0, 20, 500)

lines = plt.plot(x, np.sin(slider.value * x))

def update_lines(change):
    plt.title('Plotting: y=sin({} * x)'.format(change.new))
    lines[0].set_data(x, np.sin(change.new * x))
    fig.canvas.draw()
    fig.canvas.flush_events()

slider.observe(update_lines, names='value')

AppLayout(
    center=fig.canvas,
    footer=slider,
    pane_heights=[0, 6, 1]
)

---
---
# Faster animations 
https://www.youtube.com/watch?v=51rCh2Do2EE

In [ ]:
%matplotlib widget

import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [ ]:
plt.rcParams['figure.figsize'] = (12,4)

In [ ]:
N = 2**7

In [ ]:
lwL, lwR = 2, 1
axRxlims = axRylims = [0, N], [0, 160]

In [ ]:
_t = np.linspace(0, N, N)
_y = (1+np.sin(5/N*2*np.pi*_t)) * (1+np.cos(2/N*2*np.pi*_t)) * (0.9*_t/N+0.3) + np.random.normal(0.1, 0.01, N)

In [ ]:
dframe = np.full(N+1, fill_value=np.nan)

In [ ]:
t = np.tile(_t, (N,1))
y = np.tile(_y, (N, 1))

In [ ]:
for i in range(N):
    t[i, i+1:] = np.nan
    y[i, i+1:] = np.nan

## Replot

In [ ]:
with plt.ioff():
    fig, (axL, axR) = plt.subplots(ncols=2, tight_layout=True)

In [ ]:
plt.ioff()
fig.suptitle("Replot")
for frame in range(N):
    axL.clear(), axR.clear()
    dframe[frame] = time.perf_counter()
    y2 = 1000 * np.diff(dframe)
    y2_avg = np.nanmean(y2)
    axL.grid()
    axL.plot(t[frame], y[frame], lw=lwL)
    
    fig.canvas.draw()
    plt.pause(1e-5)
plt.ion()

## Partial Replot

In [ ]:
with plt.ioff():
    fig, (axL, axR) = plt.subplots(ncols=2, tight_layout=True)
axL.grid()
lineL = axL.plot([0,1],[0,1])[0]

In [ ]:
plt.ioff()
fig.suptitle("Partial Replot")
for frame in range(N):
    dframe[frame] = time.perf_counter()
    y2 = 1000 * np.diff(dframe)
    y2_avg = np.nanmean(y2)
    # if lineL is None:
    #     lineL = axL.plot(t[frame], y[frame], lw=lwL)[0]
    # else:
    lineL.set_data(t[frame], y[frame])
    axL.autoscale_view(True, True)
    axL.relim()
    
    fig.canvas.draw()
    plt.pause(1e-7)
plt.ion()

## Standard Func animation

In [ ]:
with plt.ioff():
    fig, (axL, axR) = plt.subplots(ncols=2, tight_layout=True)
axL.grid()
axR.grid()
lineL = axL.plot([0,1],[0,1])[0]
lineR = axR.plot([0,1],[0,1])[0]

In [ ]:
def update(frame, frame_times):
    frame_times[frame] = time.perf_counter()
    dframe[frame] = time.perf_counter()
    y2 = 1000 * np.diff(dframe)
    y2_avg = np.nanmean(y2)

    lineL.set_data(t[frame], y[frame])
    lineR.set_data(t[frame], y2)
    
    fig.canvas.draw()
    plt.pause(1e-5)

In [ ]:
ani = FuncAnimation(fig, update, interval=0, fargs=(dframe,), repeat=False, frames=list(range(N)))

In [ ]:
ani

## Trying something

In [ ]:
with plt.ioff():
    fig, (axL, axR) = plt.subplots(ncols=2, tight_layout=True)
axL.grid()
lineL = axL.plot([0,1],[0,1])[0]
axL.set_ylim([0, 3])
axL.set_xlim([0, 75])

In [ ]:
plt.ioff()
fig.suptitle("Custom")
for frame in range(N):
    update_axes = redraw = False
    lineL.set_data(t[frame], y[frame])

    if frame >= axL.get_xlim()[1]:
        update_axes = True
        redraw = True
    
    if update_axes:
        axL.set_xlim([0, int(1.5*axL.get_xlim()[1])])
    # if redraw:
    fig.canvas.draw()
    plt.pause(1e-5)
plt.ion()

---
---
# graph.py

In [ ]:
%matplotlib widget
from inference.graph import ProbabilityPlotter

In [ ]:
plotter = ProbabilityPlotter(list(range(0,10)))

In [ ]:
plotter